<h2>これはASAのACLをテストするためのノートブックです。</h2>
始めに、以下のセルでBatfishに対する質問を読み込みます。

In [2]:
%run startup.py
load_questions()

次に、以下のセルでshow running-configファイルが格納されている場所を指定します。

In [3]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "ASA_acl"
SNAPSHOT_NAME = "ASA_firewall"

SNAPSHOT_PATH = "networks/example-filters/mycurrent"

# Now create the network and initialize the snapshot
bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


'ASA_firewall'

In [3]:
parse_status = bfq.fileParseStatus().answer().frame()

In [4]:
parse_status

,File_Name,Status,Nodes
0,configs/ASA_after_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_after_acl_summary']
1,configs/ASA_before_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_before_acl_summary']
2,configs/N9kv-1.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-1']
3,configs/N9kv-2.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-2']
4,configs/after_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['after_summary_asa']
5,configs/before_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['before_summary_asa']
6,configs/ciscoasa.cfg,PARTIALLY_UNRECOGNIZED,['ciscoasa']


<h3>以下のセルでサマライズされる前のアクセスリスト のテストを行います。</h3>

In [4]:
src_list = ['172.20.0.21/16', '192.168.0.1/8', '10.20.0.21/24']
for src in src_list:
    ip_flow = HeaderConstraints(srcIps=src,
                                              dstIps='0.0.0.0')
    answer = bfq.testFilters(headers=ip_flow,
                         nodes='asa_before_acl_summary',
                         filters="SPLIT-ACL").answer() 
    show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_before_acl_summary,SPLIT-ACL,Start Location: asa_before_acl_summarySrc IP: 172.20.0.0Src Port: 49152Dst IP: 0.0.0.0Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_before_acl_summary,SPLIT-ACL,Start Location: asa_before_acl_summarySrc IP: 192.0.0.0Src Port: 49152Dst IP: 0.0.0.0Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_before_acl_summary,SPLIT-ACL,Start Location: asa_before_acl_summarySrc IP: 10.20.0.0Src Port: 49152Dst IP: 0.0.0.0Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


<h3>以下のセルでサマライズされた後のアクセスリスト のテストを行います。</h3>
サマライズ前後で結果が同じかどうか比較してみてください。

In [5]:
src_list = ['172.20.1.1/16','10.0.0.0/8']
for src in src_list:
    ip_flow = HeaderConstraints(srcIps=src,
                                                  dstIps='0.0.0.0',
                                                 applications=['dns'])
    answer = bfq.testFilters(headers=ip_flow,
                             nodes='asa_after_acl_summary',
                             filters="SPLIT-ACL").answer() 
    show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_after_acl_summary,SPLIT-ACL,Start Location: asa_after_acl_summarySrc IP: 172.20.0.0Src Port: 49152Dst IP: 0.0.0.0Dst Port: 53IP Protocol: UDP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_after_acl_summary,SPLIT-ACL,Start Location: asa_after_acl_summarySrc IP: 10.0.0.0Src Port: 49152Dst IP: 0.0.0.0Dst Port: 53IP Protocol: UDP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


In [6]:
import os
import sys
import re
#textfsmのパスをnotebookに渡す
sys.path.append('/Users/lisago/python/ansible/venv/lib/python3.7/site-packages')

import textfsm

acl_file = 'networks/example-filters/mycandidate/config/ASA_show_access_list.cfg'
ntc_template = 'ntc-templates/cisco_asa_show_access-list.template'
#acl_file = 'networks/example-filters/mycandidate/config/nxos_acl_sample.cfg'
#ntc_template = 'ntc-templates/cisco_nxos_show_access-lists.template'
#interface_config_file = 'networks/example-filters/mycandidate/config/cisco_ios_show_ip_interfaces.cfg'
#ntc_template = 'ntc-templates/cisco_ios_show_ip_interfaces.template'

'''
[{'INTF': 'Serial0', 'LINK_STATUS': 'down', 'PROTOCOL_STATUS': 'down', 'IPADDR': ['192.168.1.1/24*'],
'MASK': ['1'], 'VRF': '', 'MTU': '1500', 'IP_HELPER': [], 'OUTGOING_ACL': '', 'INBOUND_ACL': ''}]
'''

def output(interface, template):
    with open(interface, 'r') as f:
        inventory_text = f.read()
    
    with open(ntc_template) as f:
        #table = textfsm.TextFSM(f)
        fsm = textfsm.TextFSM(f)
        fsm_results = fsm.ParseText(inventory_text)
        parsed = [dict(zip(fsm.header, row)) for row in fsm_results]
        f.close()
    return parsed

def extract_interface(parsed):
    address_list = []
    for con in parsed:
        ipaddress = con['IPADDR']
        for add in ipaddress:
            #address.strip('*')
            address = re.sub(r'\*', ' ', add)
            address_list.append(address)
            
    return address_list
    

parsed = output(acl_file, ntc_template)
address_list = extract_interface(parsed)

for address in address_list:
    ip_flow = HeaderConstraints(srcIps=address)
                             #dstIps='255.255.255.0')
    answer = bfq.testFilters(headers=ip_flow,
                             nodes='before_summary_asa',
                             filters="SPLIT-ACL").answer() 
    show(answer.frame())

TextFSMError: Error: "Did not match any rules". Rule Line: 68. Input Line: access-list cached ACL log flows: total 0, denied 0 (deny-flow-max 4096).